In [1]:
from utils.datasets import *
from utils.utils import *
from models import *
from utils import torch_utils
import numpy as np

In [2]:
device = torch_utils.select_device(device='0')

Using CUDA device0 _CudaDeviceProperties(name='Quadro K620', total_memory=2048MB)



In [3]:
source=r'C:\Development\dev_tools\tree-detector-yolo\data\t1f_22.tif'

In [17]:
cfg=r'C:\\Users\\lerryw\\Google Drive\\acacia_dataset\\cfg\\yolov3-spp-1cls.cfg'
weights=r'C:\\Users\\lerryw\\Google Drive\\acacia_dataset\\weights\\last_1cls_1300.pt'

# Initialize model
model = Darknet(cfg, 416)

if weights.endswith('.pt'):  # pytorch format
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
else:  # darknet format
    load_darknet_weights(model, weights)
    
model.to(device).eval()
# Half precision
half=True
half = half and device.type != 'cpu'  # half precision only supported on CUDA
if half:
    model.half()

Model Summary: 225 layers, 6.25733e+07 parameters, 6.25733e+07 gradients


In [6]:
# dataset = LoadImages(source, img_size=1024)
dataset=ImageFolder(source, batch_size=1, img_size=416)
dataset2=LoadImages(source,img_size=416)

In [15]:
## Test dataset1
for batch_i, (img_paths, img) in enumerate(dataset):
    print(batch_i, (img_paths, img.shape))

0 (['C:\\Development\\dev_tools\\tree-detector-yolo\\data\\t1f_22.tif'], (3, 666, 666))


In [24]:
for path, img, im0s, vid_cap in dataset2:
    print(path, img.shape, im0s.shape, vid_cap)
    
    preds = []
    length = 416
    ni = int(math.ceil(im0s.shape[1] / length))  # up-down
    nj = int(math.ceil(im0s.shape[2] / length))  # left-right
    print("ni:",ni,"nj:",nj)
    for i in range(ni):
        print('row %g/%g: ' % (i, ni), end='')
        for j in range(nj):
            print('%g ' % j, end='', flush=True)
            
            y2 = min((i + 1) * length, im0s.shape[1])
            y1 = y2 - length
            x2 = min((j + 1) * length, im0s.shape[2])
            x1 = x2 - length
            print(x1,x2,y1,y2)
#             with torch.no_grad():
#                 chip = torch.from_numpy(img[:, y1:y2, x1:x2]).unsqueeze(0).to(device)
#                 pred = model(chip)

image 1/1 C:\Development\dev_tools\tree-detector-yolo\data\t1f_22.tif: C:\Development\dev_tools\tree-detector-yolo\data\t1f_22.tif (3, 416, 416) (666, 666, 3) None
ni: 2 nj: 1
row 0/2: 0 -413 3 0 416
row 1/2: 0 -413 3 250 666


In [6]:
class_name=r'C:\\Users\\lerryw\\Google Drive\\acacia_dataset\\acacia.names'
names = load_classes(class_name)
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

In [ ]:
t0 = time.time()
img = torch.zeros((1, 3, 416, 416), device=device)
_ = model(img.half() if half else img.float()) if device.type != 'cpu' else None
total_predicted_box = list()
class_list = list()
min_wh, max_wh = 2, 4096 
for batch_i, (path, img) in enumerate(dataset):
    print('\n', path, img.shape, end=' ')
    img_ud = np.ascontiguousarray(np.flip(img, axis=1))
    img_lr = np.ascontiguousarray(np.flip(img, axis=2))
    
    preds = []
    length = 416
    ni = int(math.ceil(img.shape[1] / length))  # up-down
    nj = int(math.ceil(img.shape[2] / length))  # left-right
    
    # scan image for each row based on slicing size
    for i in range(ni):  # for i in range(ni - 1):
        print('\nrow %g/%g: ' % (i, ni), end='')
        for j in range(nj):  # for j in range(nj if i==0 else nj - 1):
            print('\n%g ' % j, end='', flush=True)

            # forward scan
            y2 = min((i + 1) * length, img.shape[1])
            y1 = y2 - length
            x2 = min((j + 1) * length, img.shape[2])
            x1 = x2 - length
#             print(y2,y1,x2,x1,'\n')
            with torch.no_grad():
                # Normal orientation
                chip = torch.from_numpy(img[:, y1:y2, x1:x2]).unsqueeze(0).to(device)
                chip = chip.half() if half else chip.float()  # uint8 to fp16/32
                chip /= 255.0  # 0 - 255 to 0.0 - 1.0
                if chip.ndimension() == 3:
                    chip = chip.unsqueeze(0)
                    
                pred = model(chip, augment=True)[0]

                if half:
                    pred = pred.float()
                    
                pred = non_max_suppression(pred, 0.7, 0.4,
                                   multi_label=False, classes=None, agnostic=True)
                
                for i, det in enumerate(pred):
                    print(len(det))

In [7]:
import os
source=r'C:\\Users\\lerryw\\Google Drive\\acacia_dataset\\ayam.tif'
base = os.path.basename(source)
name =  os.path.splitext(base)
filename = os.path.splitext(os.path.basename(source))[0]
print(base)

ayam.tif
